In [ ]:
# imports
# suppress holoviews warning. Using warnings module didn't work
import logging
logging.getLogger("param.Dimension").setLevel(logging.CRITICAL)
logging.getLogger("param.ParameterizedMetaclass").setLevel(logging.CRITICAL)
logging.getLogger("param.HistogramPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.AdjointLayout").setLevel(logging.CRITICAL)
logging.getLogger("param.OverlayPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.HoloMap").setLevel(logging.CRITICAL)
logging.getLogger("param.CurvePlot").setLevel(logging.CRITICAL)
logging.getLogger("param.Layout").setLevel(logging.CRITICAL)
logging.getLogger("param.LayerPlot").setLevel(logging.CRITICAL)
logging.getLogger("param.RasterPlot").setLevel(logging.CRITICAL)
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))


import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE

import paths
import functions_bondjango as bd
import functions_misc as fm
import functions_plotting as fp
import pandas as pd
import numpy as np
import sklearn.mixture as mix
import sklearn.decomposition as decomp
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import svm, datasets
from sklearn import preprocessing
import sklearn.linear_model as lin
import sklearn.metrics as smet
import scipy.signal as ss

import random
import functions_data_handling as fd
import importlib
import processing_parameters
import h5py


In [ ]:
# Load the desired files
importlib.reload(processing_parameters)
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # define the search string
    search_string = processing_parameters.search_string
    # query the database for data to plot
    data_all = bd.query_database('analyzed_data', search_string)
    data_path = [el['analysis_path'] for el in data_all if 'combinedanalysis' in el['slug']]
#     print(data_path)
    # load the calcium data
    coefficients = []
    coefficients_shuffle = []
    r2 = []
    r2_shuffle = []
    # for all the variables
    for variables in processing_parameters.variable_title[:1]:
        # for all the files
        for files in data_path:
#             print(files)
            # load the data
            with h5py.File(files, 'r') as f:
                if 'no_ROIs' in f.keys():
                    continue
#                 print(f.keys())
                for key in f.keys():
                    
                    coefficients.append(np.array(f['regression']['coefficients_'+variables+'_']))
                    coefficients_shuffle.append(np.array(f['regression']['coefficients_'+variables+'_shuffle']))
                    r2.append(np.array(f['regression']['r2_'+variables+'_']))
                    r2_shuffle.append(np.array(f['regression']['r2_'+variables+'_shuffle']))
    #         with pd.HDFStore(files) as h:
    #             beh_data.append(h['full_traces'])
    #             if '/matched_calcium' in h.keys():
    #                 data.append(h['matched_calcium'])
    # concatenate the data
#     data = pd.concat(data)
#     print(coefficients)
    coefficients = np.concatenate(coefficients, axis=1)
    coefficients_shuffle = np.concatenate(coefficients_shuffle, axis=1)
# [print(el.shape) for el in r2]
# print(np.array(r2))

In [ ]:
# plot the r-squared values
freqR, edgesR = np.histogram(r2, 20)
# freqS, edgesS = np.histogram(r2_shuffle, 20)
idx = np.digitize(r2_shuffle, edgesR)
freqS = np.bincount(idx, minlength=len(edgesR))

real_bars = hv.Histogram((edgesR, freqR), kdims=['R2'], vdims=['Counts']).opts(xrotation=45, alpha=0.3)
shuffle_bars = hv.Histogram((edgesR, freqS), kdims=['R2'], vdims=['Counts']).opts(xrotation=45, alpha=0.3)

real_bars*shuffle_bars

In [ ]:
# plot the weight distributions
# plot the r-squared values
freqR, edgesR = np.histogram(coefficients[0, :], 20)
# freqS, edgesS = np.histogram(r2_shuffle, 20)
idx = np.digitize(coefficients_shuffle[0, :], edgesR)
freqS = np.bincount(idx, minlength=len(edgesR))

real_bars = hv.Histogram((edgesR, freqR), kdims=['R2'], vdims=['Counts']).opts(xrotation=45, alpha=0.3)
shuffle_bars = hv.Histogram((edgesR, freqS), kdims=['R2'], vdims=['Counts']).opts(xrotation=45, alpha=0.3)

real_bars*shuffle_bars